In [1]:
# pip install google-api-python-client
import googleapiclient.discovery
import pandas as pd

api_service_name = "youtube"
api_version = "v3"
DEVELOPER_KEY = "xxx"

youtube = googleapiclient.discovery.build(
    api_service_name, api_version, developerKey=DEVELOPER_KEY)

request = youtube.commentThreads().list(
    part="snippet",
    videoId="WNrB1Q9Rry0",
    maxResults=100
)
response = request.execute()

comments = []

for item in response['items']:
    comment = item['snippet']['topLevelComment']['snippet']
    comments.append([
        comment['authorDisplayName'],
        comment['publishedAt'],
        comment['updatedAt'],
        comment['likeCount'],
        comment['textDisplay']
    ])

df = pd.DataFrame(comments, columns=['author', 'published_at', 'updated_at', 'like_count', 'text'])


df.head(10)

C:\Users\poude_9elal6v\AppData\Local\Temp\ipykernel_21232\1984450032.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


,author,published_at,updated_at,like_count,text
0,@LeilaGharani,2024-01-25T17:16:47Z,2024-01-25T17:16:47Z,0,"<a href=""UCJtUOos_MwJa_Ewii-R3cJA/INVYZKnjF56F..."
1,@adarshjha4774,2024-02-13T16:06:47Z,2024-02-13T16:06:47Z,0,All these options in insert are not coming in ...
2,@truth5119,2023-12-29T07:35:07Z,2023-12-29T07:35:21Z,0,I materials had code if i want both displayed ...
3,@michaeljarcher,2023-11-24T12:17:39Z,2023-11-24T12:17:39Z,0,About time excel got with the picture. But fir...
4,@onewhitewolf13,2023-11-01T06:58:12Z,2023-11-01T06:58:12Z,1,Was this a phased release? My images are showi...
5,@md.mahmudulalamsymon1216,2023-10-18T18:12:05Z,2023-10-18T18:12:05Z,0,Wow!!<br>Thanks a lot for sharing these nice t...
6,@iditenahui,2023-09-23T18:50:10Z,2023-09-23T18:50:10Z,0,This is super helpful and useful as well! Than...
7,@siryoneyal,2023-09-06T09:13:18Z,2023-09-06T09:13:18Z,0,"@LeilaGharani , I have kind of a challenge for..."
8,@kn4golf,2023-09-05T14:03:32Z,2023-09-05T14:03:32Z,0,"As always, great presentation 🙂 Works for me i..."
9,@walterf6763,2023-09-04T21:57:03Z,2023-09-04T21:57:03Z,0,Thanks for the class. Can the same be done wi...


In [2]:
import numpy as np
import joblib
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.corpus import stopwords
import string


In [5]:
import numpy as np
import joblib
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.corpus import stopwords
import string

# Define the MyBinaryRelevance class
class MyBinaryRelevance:
    def __init__(self, classifier, max_iterations=10):
        self.classifier = classifier
        self.models = []
        self.max_iterations = max_iterations
        self.history = {"loss": []} # Initialize history dictionary

    def fit(self, X_train, Y_train):
        num_labels = Y_train.shape[1]

        for i in range(num_labels):
            model = self.classifier(kernel='linear')
            model.fit(X_train, Y_train[:, i])
            self.models.append(model)

            if i + 1 == self.max_iterations:
                break

    def predict(self, X_test):
        predictions = np.zeros((X_test.shape[0], len(self.models)))

        for i, model in enumerate(self.models):
            predictions[:, i] = model.predict(X_test)

        return predictions

# Load the saved model
loaded_classifier = joblib.load("svm_model.joblib")

# Load the CountVectorizer used for training
count_vector = joblib.load("count_vector.joblib")

# Function to preprocess a comment
def preprocess_comment(comment):
    # Remove punctuation and convert to lowercase
    comment = comment.lower().translate(str.maketrans('', '', string.punctuation))

    # Remove stop words and apply stemming and lemmatization
    stop_words = set(stopwords.words('english'))
    lemmatizer = WordNetLemmatizer()
    stemmer = PorterStemmer()
    word_list = []
    for word in comment.split():
        if word not in stop_words:
            word_list.append(stemmer.stem(lemmatizer.lemmatize(word, pos="v")))
    return " ".join(word_list)

# Assuming 'df' is your DataFrame containing the comments
for index, row in df.head(20).iterrows():
    comment = row['text']
    preprocessed_comment = preprocess_comment(comment)
    comment_vectorized = count_vector.transform([preprocessed_comment]).toarray()
    predictions_svm = loaded_classifier.predict(comment_vectorized)

    # Display the author and classification results
    print(f"Author: {row['author']}")
    label_plot = ['toxic', 'obscene', 'insult']
    for i, label in enumerate(label_plot):
        print(f"{label}: {predictions_svm[0, i]}")
    print("\n")



Author: @LeilaGharani
toxic: 0.0
obscene: 0.0
insult: 0.0


Author: @adarshjha4774
toxic: 0.0
obscene: 0.0
insult: 0.0


Author: @truth5119
toxic: 0.0
obscene: 0.0
insult: 0.0


Author: @michaeljarcher
toxic: 0.0
obscene: 0.0
insult: 0.0


Author: @onewhitewolf13
toxic: 0.0
obscene: 0.0
insult: 0.0


Author: @md.mahmudulalamsymon1216
toxic: 0.0
obscene: 0.0
insult: 0.0


Author: @iditenahui
toxic: 0.0
obscene: 0.0
insult: 0.0


Author: @siryoneyal
toxic: 0.0
obscene: 0.0
insult: 0.0


Author: @kn4golf
toxic: 0.0
obscene: 0.0
insult: 0.0


Author: @walterf6763
toxic: 0.0
obscene: 0.0
insult: 0.0


Author: @FutureCommentary1
toxic: 0.0
obscene: 0.0
insult: 0.0


Author: @m_stedt
toxic: 0.0
obscene: 0.0
insult: 0.0


Author: @wr3661
toxic: 0.0
obscene: 0.0
insult: 0.0


Author: @SilverVillacacan
toxic: 0.0
obscene: 0.0
insult: 0.0


Author: @RayMedina
toxic: 0.0
obscene: 0.0
insult: 0.0


Author: @gui.liraaa
toxic: 0.0
obscene: 0.0
insult: 0.0


Author: @2000sunsunny
toxic: 0.0
obscene: 0